In [1]:
%load_ext autoreload
%autoreload 2

import asyncio, copy, os, socket, sys, time
from functools import partial
from multiprocessing import Pool, Process
from pathlib import Path
from tqdm import tqdm

import torch
from torch import optim
from torch.utils.tensorboard import SummaryWriter

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
from libs import agg, data, fl, log, ltbl, nn, plot, poison, resnet, sim, wandb
from cfgs.fedargs import *

In [2]:
project = 'fl-poison-comp'
name = 'fg-bn-mnist-attack-fn-40'

fedargs.agg_rule = agg.Rule.FoolsGold
fedargs.dataset = "mnist"

# FLTrust
FLTrust = {"is": False,
           "ratio": 0.003,
           "data": None,
           "loader": None,
           "proxy": {"is": False,
                     "ratio": 0.5,
                     "data": None,
                     "loader": None}}

cosine_attack = {"is": False,
                 "args": {"poison_percent": 1,
                          "scale_dot": 5,
                          "scale_dot_factor": 1,
                          "scale_norm": 500,
                          "scale_norm_factor": 2,
                          "scale_epoch": 5},
                 "kn": poison.Knowledge.IN,
                 "func": poison.sine_attack}


# Fang attack proposed for trimmed mean
fang_attack = {"is": False,
               "kn": poison.Knowledge.PN,
               "func": poison.fang_trmean}

# LIE attack
lie_attack = {"is": False,
              "kn": poison.Knowledge.PN,
              "func": poison.lie_attack}

# SOTA attack proposed for trimmed mean, min-max, min-sum
sota_attack = {"is": False,
               "kn": poison.Knowledge.PN,
               "dev_type": "unit_vec", # see other possible functions in poison python file
               "func": poison.sota_agnostic_min_max # see other possible functions in poison python file
               }

# No of malicious clients
mal_clients = [c for c in range(20)]

# Save Logs To File (info | debug | warning | error | critical) [optional]
log.init("info")
#log.init("info", name)
#log.init("debug", name)

fedargs.tb = SummaryWriter('../out/runs/' + project + '/' + name, comment="fl")
plot = plot.init(name, project)
data_dict = {'epochs':[],'acc': [], 'test_loss': [], 'asr': [],'bsr':[]}
ltbl = ltbl.init(data_dict, name, project)
wb = wandb.init(name, project)

wandb: Currently logged in as: kasyah (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [3]:
# Device settings
use_cuda = fedargs.cuda and torch.cuda.is_available()
torch.manual_seed(fedargs.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

In [4]:
# Prepare clients
host = socket.gethostname()
clients = [host + "(" + str(client + 1) + ")" for client in range(fedargs.num_clients)]

In [5]:
# Initialize Global and Client models
global_model = copy.deepcopy(fedargs.model)
# Load Data to clients
train_data, test_data = data.load_dataset(fedargs.dataset)

<h2>FLTrust</h2>

In [6]:
if FLTrust["is"]:
    train_data, FLTrust["data"] = data.random_split(train_data, FLTrust["ratio"])
    FLTrust["loader"] = torch.utils.data.DataLoader(FLTrust["data"], batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)
    
    if FLTrust["proxy"]["is"]:
        FLTrust["data"], FLTrust["proxy"]["data"] = data.random_split(FLTrust["data"], FLTrust["proxy"]["ratio"])
        FLTrust["loader"] = torch.utils.data.DataLoader(FLTrust["data"], batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)
        FLTrust["proxy"]["loader"] = torch.utils.data.DataLoader(FLTrust["proxy"]["data"], batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)

<h2>Prepare a backdoored loader for test</h2>

In [7]:
if backdoor_attack["is"]:
    train_data, backdoor_attack["data"] = data.random_split(train_data, backdoor_attack["ratio"])
    backdoor_attack["data"] = poison.insert_trojan(backdoor_attack["data"],
                                                   backdoor_attack["target_label"],
                                                   backdoor_attack["trojan_func"], 1)
    backdoor_attack["loader"] = torch.utils.data.DataLoader(backdoor_attack["data"], batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)

<h2>Load client's data</h2>

In [8]:
clients_data = data.split_data(train_data, clients)

<h2>Label Flip Attack</h2>

In [9]:
if label_flip_attack["is"]:
    for client in mal_clients:
        clients_data[clients[client]] = label_flip_attack["func"](clients_data[clients[client]],
                                                                  label_flip_attack["labels"],
                                                                  label_flip_attack["percent"])

<h2>Backdoor Attack</h2>

In [10]:
if backdoor_attack["is"]:
    for client in mal_clients:
        clients_data[clients[client]] = poison.insert_trojan(clients_data[clients[client]],
                                                             backdoor_attack["target_label"],
                                                             backdoor_attack["trojan_func"], 0.5)

In [11]:
client_train_loaders, _ = data.load_client_data(clients_data, fedargs.client_batch_size, None, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=fedargs.test_batch_size, shuffle=True, **kwargs)

client_details = {
        client: {"train_loader": client_train_loaders[client],
                 "model": copy.deepcopy(global_model),
                 "model_update": None}
        for client in clients
    }

In [12]:
def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)

    return wrapped

@background
def process(client, epoch, model, train_loader, fedargs, device):
    # Train
    model_update, model, loss = fedargs.train_func(model, train_loader, 
                                                   fedargs.learning_rate,
                                                   fedargs.weight_decay,
                                                   fedargs.local_rounds, device)

    log.jsondebug(loss, "Epoch {} of {} : Federated Training loss, Client {}".format(epoch, fedargs.epochs, client))
    log.modeldebug(model_update, "Epoch {} of {} : Client {} Update".format(epoch, fedargs.epochs, client))
    
    return model_update

In [13]:
import time
start_time = time.time()
    
# Federated Training
for epoch in tqdm(range(fedargs.epochs)):
    log.info("Federated Training Epoch {} of {}".format(epoch, fedargs.epochs))

    # Global Model Update
    if epoch > 0:
        # For Tmean and FLTrust, not impacts others as of now
        avgargs = {"beta": len(mal_clients), 
                   "base_model_update": global_model_update if FLTrust["is"] else None,
                   "base_norm": True}
        
        # Average
        global_model = fl.federated_avg(client_model_updates, global_model, fedargs.agg_rule, **avgargs)
        log.modeldebug(global_model, "Epoch {}: Server Update".format(epoch))
        
        # Test, Plot and Log
        global_test_output = fedargs.eval_func(global_model, test_loader, device, label_flip_attack["labels"])
        fedargs.tb.add_scalar("Gloabl Accuracy/", global_test_output["accuracy"], epoch)
        fedargs.tb.add_scalar("Global Test Loss/", global_test_output["test_loss"], epoch)
        plot.log({epoch: {"time": time.time(), "acc": global_test_output["accuracy"], "loss": global_test_output["test_loss"]}})
        wb.log({"epoch": epoch, "time": time.time(), "acc": global_test_output["accuracy"], "loss": global_test_output["test_loss"]})
        log.jsoninfo(global_test_output, "Global Test Outut after Epoch {} of {}".format(epoch, fedargs.epochs))
        
        # Evaluate LFA
        if "attack" in global_test_output:
            if "attack_success_rate" in global_test_output["attack"]:
                fedargs.tb.add_scalar("Attack Success Rate/", global_test_output["attack"]["attack_success_rate"], epoch)
                plot.alog(str(epoch), {"attack_success_rate": global_test_output["attack"]["attack_success_rate"]})
                wb.log({"attack_success_rate": global_test_output["attack"]["attack_success_rate"]})
            if "misclassification_rate" in global_test_output["attack"]:
                fedargs.tb.add_scalar("Misclassification Rate/", global_test_output["attack"]["misclassification_rate"], epoch)
                plot.alog(str(epoch), {"misclassification_rate": global_test_output["attack"]["misclassification_rate"]})
                wb.log({"misclassification_rate": global_test_output["attack"]["misclassification_rate"]})

        # Evaluate Backdoor
        if backdoor_attack["is"]:
            backdoor_test_output = fl.backdoor_test(global_model, backdoor_attack["loader"], device, backdoor_attack["target_label"])
            fedargs.tb.add_scalar("Backdoor Success Rate/", backdoor_test_output["accuracy"], epoch)
            plot.alog(str(epoch), {"backdoor_success_rate": backdoor_test_output["accuracy"]})
            wb.log({"backdoor_success_rate": backdoor_test_output["accuracy"]})
            log.jsoninfo(backdoor_test_output, "Backdoor Test Outut after Epoch {} of {}".format(epoch, fedargs.epochs))

        # CSV logging
        ltbl.ar({'epochs': epoch,'acc': global_test_output["accuracy"], 'test_loss': global_test_output["test_loss"]})

        # Update client models
        for client in clients:
            client_details[client]['model'] = copy.deepcopy(global_model)

    # Clients
    tasks = [process(client, epoch, client_details[client]['model'],
                     client_details[client]['train_loader'],
                     fedargs, device) for client in clients]
    try:
        updates = fedargs.loop.run_until_complete(asyncio.gather(*tasks))
    except KeyboardInterrupt as e:
        log.error("Caught keyboard interrupt. Canceling tasks...")
        tasks.cancel()
        fedargs.loop.run_forever()
        tasks.exception()

    for client, update in zip(clients, updates):
        client_details[client]['model_update'] = update
    client_model_updates = {client: details["model_update"] for client, details in client_details.items()}
    
    # Fang attack
    if fang_attack["is"]:
        client_model_updates = fang_attack["func"](client_model_updates, len(mal_clients), fang_attack["kn"])
        
    # LIE attack
    if lie_attack["is"]:
        client_model_updates = lie_attack["func"](client_model_updates, len(mal_clients), lie_attack["kn"])
   
    # SOTA attack
    if sota_attack["is"]:
        client_model_updates = sota_attack["func"](client_model_updates, len(mal_clients), 
                                                   sota_attack["kn"], sota_attack["dev_type"])
    
    # FLtrust or FLTC based aggregation rules or attacks
    if FLTrust["is"]:
        global_model_update, _, _ = fedargs.train_func(global_model, FLTrust["loader"],
                                                       fedargs.learning_rate,
                                                       fedargs.weight_decay,
                                                       fedargs.local_rounds, device)

        # For Attacks related to FLTrust
        base_model_update = global_model_update
        if FLTrust["proxy"]["is"]:
            base_model_update, _, _ = fedargs.train_func(global_model, FLTrust["proxy"]["loader"],
                                                         fedargs.learning_rate,
                                                         fedargs.weight_decay,
                                                         fedargs.local_rounds, device)
        
        # Layer replacement attack
        if layer_replacement_attack["is"]:
            for client in mal_clients:
                client_details[clients[client]]['model_update'] = layer_replacement_attack["func"](base_model_update,
                                                                                                   client_details[clients[client]]['model_update'],
                                                                                                   layer_replacement_attack["layers"])

        # For cosine attack, Malicious Clients
        if cosine_attack["is"]:
            p_models, params_changed = cosine_attack["func"](base_model_update, cosine_attack["args"], epoch,
                                                             client_model_updates, len(mal_clients), cosine_attack["kn"])
            
            for client, p_model in enumerate(p_models):
                client_details[clients[client]]['model_update'] = p_model 

            #plot params changed for only one client
            fedargs.tb.add_scalar("Params Changed for Cosine Attack/", params_changed, epoch)

        # For sybil attack, Malicious Clients
        if sybil_attack["is"]:
            for client in mal_clients:
                client_details[clients[client]]['model_update'] = base_model_update
                
        # again pair, as changed during attack
        client_model_updates = {client: details["model_update"] for client, details in client_details.items()}

print(time.time() - start_time)

  2%|▏         | 1/51 [00:41<34:21, 41.23s/it]2021-11-24 16:01:38,844 - <ipython-input-13-73d92767348d>::<module>(l:6) : Federated Training Epoch 1 of 51 [MainProcess : MainThread (INFO)]
2021-11-24 16:01:43,904 - <ipython-input-13-73d92767348d>::<module>(l:25) : Global Test Outut after Epoch 1 of 51 {
    "accuracy": 75.14999999999999,
    "correct": 7515,
    "test_loss": 1.9930617408752442
} [MainProcess : MainThread (INFO)]
  4%|▍         | 2/51 [01:27<36:19, 44.48s/it]2021-11-24 16:02:25,587 - <ipython-input-13-73d92767348d>::<module>(l:6) : Federated Training Epoch 2 of 51 [MainProcess : MainThread (INFO)]
2021-11-24 16:02:30,657 - <ipython-input-13-73d92767348d>::<module>(l:25) : Global Test Outut after Epoch 2 of 51 {
    "accuracy": 81.98,
    "correct": 8198,
    "test_loss": 1.4150715751647949
} [MainProcess : MainThread (INFO)]
  6%|▌         | 3/51 [02:14<36:22, 45.46s/it]2021-11-24 16:03:12,237 - <ipython-input-13-73d92767348d>::<module>(l:6) : Federated Training Epoch 3 

 39%|███▉      | 20/51 [15:36<24:19, 47.08s/it]2021-11-24 16:16:34,484 - <ipython-input-13-73d92767348d>::<module>(l:6) : Federated Training Epoch 20 of 51 [MainProcess : MainThread (INFO)]
2021-11-24 16:16:39,238 - <ipython-input-13-73d92767348d>::<module>(l:25) : Global Test Outut after Epoch 20 of 51 {
    "accuracy": 93.54,
    "correct": 9354,
    "test_loss": 0.22707710909843445
} [MainProcess : MainThread (INFO)]
 41%|████      | 21/51 [16:23<23:24, 46.82s/it]2021-11-24 16:17:20,712 - <ipython-input-13-73d92767348d>::<module>(l:6) : Federated Training Epoch 21 of 51 [MainProcess : MainThread (INFO)]
2021-11-24 16:17:26,065 - <ipython-input-13-73d92767348d>::<module>(l:25) : Global Test Outut after Epoch 21 of 51 {
    "accuracy": 93.73,
    "correct": 9373,
    "test_loss": 0.22143061666488648
} [MainProcess : MainThread (INFO)]
 43%|████▎     | 22/51 [17:11<22:50, 47.27s/it]2021-11-24 16:18:09,035 - <ipython-input-13-73d92767348d>::<module>(l:6) : Federated Training Epoch 22 of

2021-11-24 16:31:33,009 - <ipython-input-13-73d92767348d>::<module>(l:25) : Global Test Outut after Epoch 39 of 51 {
    "accuracy": 95.98,
    "correct": 9598,
    "test_loss": 0.13762558525204657
} [MainProcess : MainThread (INFO)]
 78%|███████▊  | 40/51 [31:17<08:38, 47.11s/it]2021-11-24 16:32:14,811 - <ipython-input-13-73d92767348d>::<module>(l:6) : Federated Training Epoch 40 of 51 [MainProcess : MainThread (INFO)]
2021-11-24 16:32:19,817 - <ipython-input-13-73d92767348d>::<module>(l:25) : Global Test Outut after Epoch 40 of 51 {
    "accuracy": 96.00999999999999,
    "correct": 9601,
    "test_loss": 0.13464320912361144
} [MainProcess : MainThread (INFO)]
 80%|████████  | 41/51 [32:04<07:51, 47.11s/it]2021-11-24 16:33:01,935 - <ipython-input-13-73d92767348d>::<module>(l:6) : Federated Training Epoch 41 of 51 [MainProcess : MainThread (INFO)]
2021-11-24 16:33:07,412 - <ipython-input-13-73d92767348d>::<module>(l:25) : Global Test Outut after Epoch 41 of 51 {
    "accuracy": 96.06,


2389.125291109085


In [14]:
nn.ModelMNIST()

ModelMNIST(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

<h1> End </h1>